# Imports

In [1]:
# import time
from technician_routing.routing_toolkit import RoutingConfig, RoutingInterface
from technician_routing.routing_toolkit.engine import create_search_parameters

# Demo

#### Configure and Load Data

In [6]:
config = RoutingConfig(
    'u:/data/darnley_tech_address.csv', 
    'u:/data/address_lat_long_121923.csv',
    40, # MPH
    55, # Minutes Per Service
    60*7 # Max Minutes per Route
)
interface = RoutingInterface(config).load_data()

Set 5269 client addresses
Using uniform minutes on location of 55
Data Loaded


#### Choose your clients

In [7]:
focus_addresses = interface.client_addresses[0:1000]
print(len(focus_addresses))

1000


#### Choose your technicians

In [8]:
#drivers = 25 # Weighted-Random Sample
drivers = [0,1,2,3,4,5,6] * 40 # Four of each technician
print(len(drivers))

280


#### Optionally Ovewrite certain defaults at runtime

In [13]:
interface \
    .set_driver_speed(45) \
    .set_addresses(focus_addresses) \
    .set_time_on_location(65) \
    #.set_search_parameters(create_search_parameters(search_time_seconds=15))

Using driver speed of 45 MPH
Set 1000 client addresses
Using uniform minutes on location of 65


#### Run

In [14]:
# Random Drivers
routes = interface.run(route_drivers=drivers)
routes

[0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6]
Calculated 1007 node durations in 1.5087647438049316
Running routing for 1007 addresses and 280 drivers


Exception: Failed to solve problem, likely too few routes/vehicles or too low of a time limit

#### Detail Routes

In [ ]:
for route_number, route in enumerate(routes):
    
    travel_time = 0
    for start,end in zip(route, route[1:]):
        travel_time += interface.solver.distances[start, end]

    site_time = 0
    for location in route[1:-1]:
        site_time += interface.solver.demands[location]

    tech = interface.df_techs.iloc[route[0], :]
    #print(tech)
     
    print(f'Route {route_number}: {int(travel_time + site_time)}','minutes')
    print(f'Technician {route[0]}: {tech["Tech"]} - {tech["address"]}')

    for stop in route[1:-1]:
        print(f'\t{stop}: {interface.last_addresses[stop]}')
    

    print(int(travel_time), 'travel minutes') 
    print(site_time, 'minutes on site')
    print('\n')